In [1]:
import pandas as pd

import sys

sys.path.append("./..")
from src.utils.get_youtube_url_official import find_urls_for_songs

In [2]:
def get_uri_for_curated(number_of_songs=4):
    df = pd.read_csv("../data/top_songs_curated.csv")
    df_missing = df[df["youtube_title"].isna()]

    df_missing_n = df_missing.head(number_of_songs)

    songs = (
        df_missing_n[["Track Name", "Artist Name(s)"]]
        .rename(columns={"Track Name": "title", "Artist Name(s)": "artist"})
        .to_dict(orient="records")
    )

    results = find_urls_for_songs(songs)
    results = pd.DataFrame.from_dict(results, orient="index")
    results = results[results["youtube_title"] != "Not found"]

    merged_df = df.merge(
        results,
        how="left",
        left_on=["Track Name", "Artist Name(s)"],
        right_on=["title", "artist"],
        suffixes=("", "_drop"),
    )

    merged_df["youtube_title"] = merged_df["youtube_title"].combine_first(
        merged_df["youtube_title_drop"]
    )
    merged_df["youtube_url"] = merged_df["youtube_url"].combine_first(
        merged_df["youtube_url_drop"]
    )
    merged_df = merged_df.drop(
        columns=["title", "artist", "youtube_title_drop", "youtube_url_drop"]
    )

    merged_df = merged_df.sort_values(["Artist Name(s)", "Track Name"])
    merged_df.to_csv("../data/top_songs_curated.csv", index=False)

In [ ]:
get_uri_for_curated(number_of_songs=100)

API key loaded successfully!
Searching for 'What Goes Around...Comes Around - Radio Edit' by Justin Timberlake...
Searching for 'All Of The Lights - Album Version (Edited)' by Kanye West...
Searching for 'Heartless' by Kanye West...
Searching for 'Jesus Walks' by Kanye West...
Searching for 'POWER - Album Version (Edited)' by Kanye West...
Searching for 'Stronger' by Kanye West...
Searching for 'Birthday' by Katy Perry...
Searching for 'Daisies' by Katy Perry...
Searching for 'Electric - Pokemon 25 Version' by Katy Perry...
Searching for 'Firework' by Katy Perry...
Searching for 'Harleys In Hawaii' by Katy Perry...
Searching for 'Hot N Cold' by Katy Perry...
Searching for 'Hummingbird Heartbeat' by Katy Perry...
Searching for 'I Kissed A Girl' by Katy Perry...
Searching for 'International Smile' by Katy Perry...
Searching for 'Last Friday Night (T.G.I.F.)' by Katy Perry...
Searching for 'Never Really Over' by Katy Perry...
Searching for 'Never Worn White' by Katy Perry...
Searching for